In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604520,32.3,1292120,69.1,1292120,69.1
Vcells,1086848,8.3,8388608,64.0,1632674,12.5


In [2]:
semilla <- 100019
set.seed(semilla)

In [3]:
#Feature Engineering
#creo nuevas variables dentro del mismo mes
#Condimentar a gusto con nuevas variables

require("data.table")

#Establezco el Working Directory
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory

kcarpeta_datasets    <- "./datasets/"   #KAGGLE
#kcarpeta_datasets    <- "./buckets/b1/datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")
karchivo_salida      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_default.csv")



#lectura rapida del dataset  usando fread  de la libreria  data.table
dataset_orig  <- fread(karchivo_entrada)



Loading required package: data.table



In [4]:
EnriquecerDataset <- function( dataset )
{
  columnas_originales <-  copy(colnames( dataset ))

  #INICIO de la seccion donde se deben hacer cambios con variables nuevas
  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , mv_status02       := Master_status +  Visa_status ]
  dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]

  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]


    
  #combino MasterCard y Visa
  dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
    
    columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )

    return (dataset)
}
#------------------------------------------------------------------------------
dataset_enriq <- EnriquecerDataset( dataset_orig )


In [5]:
as.data.table(sapply(dataset_enriq[, list(mttarjeta_visa_debitos_automaticos)], sample, 10))

mttarjeta_visa_debitos_automaticos
<dbl>
2613.97
2985.28
10085.93
4852.26
36830.10
6014.69
10374.01
0.00
0.00


In [6]:
head(dataset_orig)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,clase_ternaria,mv_status01,mv_status02,mv_status03,mv_status04,mv_status05,mv_status06,mv_status07,mv_Fvencimiento,mv_Finiciomora
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
31115668,202101,1,0,0,56,132,1897.12,24324.75,940.32,⋯,CONTINUA,0,0,0,0,0,0,0,-2218,NA
31115678,202101,1,0,0,48,100,-384.72,33860.45,-515.53,⋯,CONTINUA,NA,NA,10,20,1010,10,10,NA,NA
31117415,202101,1,0,0,60,53,-3622.66,-21310.32,2442.04,⋯,CONTINUA,0,0,0,0,0,0,0,-1733,NA
31117711,202101,1,0,0,46,273,14542.05,125727.99,1037.77,⋯,CONTINUA,0,0,0,0,0,0,0,-1307,NA
31118145,202101,1,0,0,47,192,1868.27,37305.29,1496.68,⋯,CONTINUA,0,0,0,0,0,0,0,-1853,NA
31118204,202101,1,0,0,68,258,2301.86,60781.98,3240.88,⋯,CONTINUA,0,NA,10,10,10,0,0,-180,NA


In [7]:
  #grabo con nombre extendido
  fwrite( dataset_enriq,
          file=karchivo_salida,
          sep= "," )